In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [5]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [6]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [7]:
# MNIST data image of shape 28 * 28 = 784
linear = torch.nn.Linear(784, 10, bias=True).to(device)

In [9]:
# Initialization
torch.nn.init.normal_(linear.weight)

Parameter containing:
tensor([[-0.3303,  0.3255, -0.4488,  ...,  0.1155,  0.4615,  0.9316],
        [-0.1027, -0.6424, -0.4032,  ...,  0.4605,  2.1825,  0.3425],
        [-0.1665,  1.5384, -0.3312,  ...,  0.7681, -1.0083,  0.6301],
        ...,
        [ 0.1069,  0.8268,  1.2518,  ..., -1.3152, -2.8423,  0.6586],
        [-0.6342,  1.2379, -1.4351,  ..., -0.0283,  0.0826, -1.2305],
        [ 0.6733, -0.7177, -0.5025,  ..., -1.5074,  0.5423, -1.2292]],
       requires_grad=True)

In [10]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(linear.parameters(), lr=learning_rate)

In [11]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # reshape input image into [batch_size by 784]
        # label is not one-hot encoded
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = linear(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 5.592440605
Epoch: 0002 cost = 1.606336832
Epoch: 0003 cost = 1.040094495
Epoch: 0004 cost = 0.819040418
Epoch: 0005 cost = 0.699814439
Epoch: 0006 cost = 0.624610245
Epoch: 0007 cost = 0.571742713
Epoch: 0008 cost = 0.531218708
Epoch: 0009 cost = 0.499184370
Epoch: 0010 cost = 0.474031568
Epoch: 0011 cost = 0.452233613
Epoch: 0012 cost = 0.434768230
Epoch: 0013 cost = 0.418995410
Epoch: 0014 cost = 0.405672163
Epoch: 0015 cost = 0.393959165
Learning finished


In [16]:
# Test the model using test sets
with torch.no_grad():
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = linear(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r + 1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = linear(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Accuracy: 0.8863000273704529
Label:  3
Prediction:  3
